In [4]:
import tensorflow as tf
from sklearn.datasets import fetch_mldata

import numpy as np

In [13]:
mnist = fetch_mldata("MNIST Original")
X, y = mnist.data.astype(np.float32) / 255., mnist.target.astype(np.int32)

index = np.arange(X.shape[0])
np.random.shuffle(index)

X, y = X[index], y[index]

print(X.shape, y.shape)

(70000, 784) (70000,)


In [16]:
X_train, X_test = X[:-10000], X[-10000:]
y_train, y_test = y[:-10000], y[-10000:]

### build model with `tf.learn`

In [19]:
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], 
                                         n_classes=10, 
                                         feature_columns=feature_cols)
dnn_clf.fit(x=X_train, y=y_train, batch_size=64, steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_evaluation_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8601922898>, 'keep_checkpoint_every_n_hours': 10000, 'keep_checkpoint_max': 5, 'save_checkpoints_steps': None, '_master': '', '_is_chief': True, 'save_checkpoints_secs': 600, 'tf_random_seed': None, '_environment': 'local', 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, 'save_summary_steps': 100, '_num_ps_replicas': 0}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and bat

In [27]:
from sklearn.metrics import accuracy_score
y_pred = list(dnn_clf.predict(X_test))
accuracy_score(y_test, y_pred)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Summary name dnn/hiddenlayer_0:fraction_of_zero_values is illegal; using

0.98399999999999999

In [28]:
dnn_clf.evaluate(X_test, y_test)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Summary name dnn/hiddenlayer_0:fraction_of_zero_values is illegal; using

{'accuracy': 0.98400003, 'global_step': 10000, 'loss': 0.054800361}

## tensorflow DNN

In [89]:
tf.reset_default_graph()

In [90]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_output = 10

X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
y = tf.placeholder(tf.int32, shape=[None], name="y")

In [91]:
def layer(name, n, x, activation=None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        W = tf.Variable(tf.truncated_normal((n_inputs, n), stddev=stddev), name="W")
        b = tf.Variable(tf.zeros([n]), name="b")
        z = tf.add(tf.matmul(x, W), b, name="z")
        if activation is None:
            return tf.nn.relu(z)
        else:
            return activation(z)

In [92]:
with tf.name_scope("dnn"):
    hidden1 = layer("hidden1", n_hidden1, X)
    hidden2 = layer("hidden2", n_hidden2, hidden1)
    logits = layer("output", n_output, hidden2, tf.identity)
    probs = tf.nn.softmax(logits)
    labels = tf.cast(tf.argmax(probs, 1), tf.int32)
    
with tf.name_scope("metrics"):
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y))
    accuracy = tf.reduce_mean(tf.cast(tf.equal(labels, y), tf.float32))

In [93]:
writer = tf.summary.FileWriter("dnn-logs")

train_op = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

init = tf.global_variables_initializer()

In [94]:
def generate_batch(x, y, batch_size=64):
    i, n = 0, x.shape[0]
    while True:
        i = i % n
        yield x[i:i+batch_size], y[i:i+batch_size]
        i += batch_size
        if i > n: i = 0
            
batches = generate_batch(X_train, y_train)

In [95]:
n_epochs = 10000

with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        x_batch, y_batch = next(batches)
        sess.run(train_op, feed_dict={X:x_batch, y:y_batch})
        if i % 100 == 0:
            test_acc = accuracy.eval(feed_dict={X:X_test, y:y_test})
            train_loss = loss.eval(feed_dict={X:x_batch, y:y_batch})
            test_loss = loss.eval(feed_dict={X:X_test, y:y_test})
#             print(train_loss)
            print("train_loss: %.2g, test_loss: %.2g, test_accuracy: %.2g" % (train_loss, test_loss, test_acc))

train_loss: 2, test_loss: 2.4, test_accuracy: 0.17
train_loss: 0.25, test_loss: 0.3, test_accuracy: 0.91
train_loss: 0.37, test_loss: 0.23, test_accuracy: 0.93
train_loss: 0.13, test_loss: 0.19, test_accuracy: 0.95
train_loss: 0.17, test_loss: 0.19, test_accuracy: 0.94
train_loss: 0.12, test_loss: 0.15, test_accuracy: 0.95
train_loss: 0.094, test_loss: 0.15, test_accuracy: 0.96
train_loss: 0.15, test_loss: 0.14, test_accuracy: 0.96
train_loss: 0.053, test_loss: 0.14, test_accuracy: 0.96
train_loss: 0.11, test_loss: 0.13, test_accuracy: 0.96
train_loss: 0.058, test_loss: 0.11, test_accuracy: 0.97
train_loss: 0.025, test_loss: 0.11, test_accuracy: 0.97
train_loss: 0.033, test_loss: 0.11, test_accuracy: 0.97
train_loss: 0.13, test_loss: 0.097, test_accuracy: 0.97
train_loss: 0.046, test_loss: 0.13, test_accuracy: 0.96
train_loss: 0.049, test_loss: 0.097, test_accuracy: 0.97
train_loss: 0.14, test_loss: 0.09, test_accuracy: 0.97
train_loss: 0.06, test_loss: 0.082, test_accuracy: 0.97
train

In [98]:
a = tf.constant([[1.1, 1.2, 1.05], [1, 1.9, 0.9]])
b = tf.constant([1, 0])
c = tf.nn.in_top_k(a, b, 1)
with tf.Session() as sess:
    print(sess.run(c))

[ True False]
